In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from IPython.display import display, HTML
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt
!pip install ffmpeg

puntos = [[22,25],[22,95],[77,60]] #Ingresar puntos de arena en formato [[eje Y,eje X],[eje Y,eje X],(...)]

##################################################################################
# EJEMPLOS EXPUESTOS EN PAPER                                                    #
#--------------------------------------------------------------------------------#
# 1 monticulos 0 colisiones (base): [[15,20]]                                    #
# 2 monticulos 1 colisiones (tangente): [[42,23],[42,97]]                        #
# 2 monticulos 1 colisiones (intermedia): [[42,25],[42,95]]                      #
# 2 monticulos 1 colisiones (intensa): [[42,36],[42,85]]                         #
# 3 monticulos 3 colisiones (intermedia): [[22,25],[22,95],[77,60]]              #
# 4 monticulos 4 colisiones (intermedia): [[50,20],[90,60],[50,100],[10,60]]     #
##################################################################################
# EJEMPLOS PROPUESTOS PARA FUTURAS INVESTIGACIONES                               #
#--------------------------------------------------------------------------------#
# 6 monticulos 6 colisiones (intermedia):                                        #
# 3 monticulos 3 colisiones (intensa):                                           #
##################################################################################

#Autoría: Ignacio Corvalán - Sebastián Menares - Matías Hermosilla - Andrés Guevara. Facultad de ing. y ciencias UAI.
#Créditos a: Nikita Yalovega por Modelo Abelian Sandpile Base https://github.com/kivyfreakt/sandpile/tree/master/sandpile

In [ ]:
#Se define la clase Sandpile, la cual contiene diferentes métodos.
class Sandpile():
    #Se define el constructor de la clase, donde además se establece el umbral critico que tendrá el algoritmo
    def __init__(self, arr=None, puntos_c=None, max_sand=4):
        #Se establece el numero maximo de granos de un monticulo
        self.max_grains = max_sand
        #Se determinan los centros de los monituculos
        self.centros = puntos_c
        # Variable para saber el ancho de la grilla
        max_distancia = 0  

        # Compara las coordenadas de anchura para ver cuál es la distancia más larga
        for i in self.centros:
            for j in i:
                # Guarda el número mayor para generar la matriz
                if (max_distancia < j):
                    max_distancia = j
        # Variable para saber el ancho de la grilla
        self.distancia = max_distancia
        #Se comprueba si no se asignó una matriz, en caso de no ser asi la define en base a la variable max_distancia
        if arr is None:
            self.rows = (max_distancia - 1) + 3
            self.cols = (max_distancia - 1) + 3
            self.grid = np.zeros(((max_distancia - 1) + 3, (max_distancia - 1) + 3), int)
        #Si se entrego una matriz, se establecen las dimensiones de esta   
        else:
            self.rows = len(arr)
            self.cols = len(arr[0])
            self.grid = np.array(arr)

    #Con este método se manejan los colapsos de los monticulos con su respectiva asignación de arena restante           
    def topple(self, elem_x, elem_y):
        p = self.grid[elem_x, elem_y]
        b = p // self.max_grains #División de granos de arena usando división entera para acelerar el proceso y disminuir iteraciones
        o = p % self.max_grains
        self.grid[elem_x, elem_y] = o

        self.grid[elem_x-1, elem_y] += b
        self.grid[elem_x+1, elem_y] += b
        self.grid[elem_x, elem_y-1] += b
        self.grid[elem_x, elem_y+1] += b


    #Con el siguiente método se adapta la grilla en caso de que exista arena que toque los bordes de la grilla
    def adaptar(self):
        existe = False
        centros = self.centros
        delta = int((self.cols - self.distancia) / 2) -1

        for filas in range(4, self.rows - 4):
            if (self.grid[filas][4] > 0) or (self.grid[filas][self.cols - 5]):
                existe = True

        for columnas in range(4, self.cols - 4):
            if (self.grid[4][columnas] > 0) or (self.grid[self.rows - 5][columnas]):
                existe = True

        if (existe == True):
            # Agregar cuatro columnas al principio y al final de la grilla
            columnas_extra_inicio = np.zeros((self.rows, 4))
            columnas_extra_final = np.zeros((self.rows, 4))
            self.grid = np.hstack((columnas_extra_inicio, self.grid, columnas_extra_final))
            self.cols = self.cols + 8

            # Agregar cuatro filas al principio y al final de la grilla
            filas_extra_inicio = np.zeros((4, self.cols))
            filas_extra_final = np.zeros((4, self.cols))
            self.grid = np.vstack((filas_extra_inicio, self.grid, filas_extra_final))
            self.rows = self.rows + 8
    #Con este método se adaptan los puntos ingresados inicialmente como centros de los monticulos. Transformandolo a la nueva grilla
    def marcar(self, puntos_c=None):
        delta = int((self.cols - self.distancia) / 2) -1 #TEST factor de corección
        centros = puntos_c
        nuevos_centros = []
        
        for centro in self.centros:
            nuevo_x = centro[1] + delta
            nuevo_y = centro[0] + delta
            nuevos_centros.append([nuevo_y, nuevo_x])
        
        return nuevos_centros
    #Este método se encarga de la ejecución del modelo hasta que cada celda tenga una cantidad igual o menor al umbral crítico, es decir hasta que el sistema se relaja
    def run(self):
        from time import time

        start_time = time()
        iterations = 0
        topple = self.topple
        adaptar = self.adaptar
        where = np.where

        while np.max(self.grid) >= self.max_grains:
            elem_x, elem_y = where(self.grid >= self.max_grains)
            topple(elem_x, elem_y)
            adaptar()
            iterations += 1

        print("--- %d iterations %s seconds ---" % (iterations, time() - start_time))
    #Método para retornar la grilla 
    def get_pile(self):
        return self.grid
    #Método para establecer la cantidad de arena en los puntos determinados 
    def set_sand(self, number):
        centros = self.centros
        
        for i in centros:
            self.grid[i[0]-1,i[1]-1] = number
    #Método para sumar mas monticulos de arena
    def __add__(self, other):
        result = Sandpile(rows = self.rows, cols = self.cols)
        try:
            result.grid = self.grid + other.grid
            return result.run()
        except ValueError:
            print("ValueError: sandpile grid sizes must match")

    #Con este método se guarda la grilla con los monticulos de arena, además de asignar colores a los diferentes niveles de arena.
    def save(self, filename="sandpile.png"):
        from PIL import Image
        colors = [(0, 0, 0), (255, 0, 0), (255, 165, 0), (255, 255, 0)]  # Colors for heat scale
        img = Image.fromarray(color_grid(self.grid, colors), "RGB")
        img.save(filename)
    #Método para encontrar el punto medio de colisión de monticulos de arenas pares(2,4,6,etc)
    def punto_medio(self, coordenadas):
        
        min_y = min(coordenadas, key=lambda c: c[0])[0]
        max_y = max(coordenadas, key=lambda c: c[0])[0]
        min_x = min(coordenadas, key=lambda c: c[1])[1]
        max_x = max(coordenadas, key=lambda c: c[1])[1]


        puntomedio_x = int((max_x + min_x)/2)
        puntomedio_y = int((max_y + min_y)/2)

        return puntomedio_y,puntomedio_x
        
    #Método para dibujar cuadrantes en la imagen de la grilla final.
    def show_with_square(self,save = False, filename = "sandpile_square.png"):
        fig,ax = plt.subplots()
        ax.imshow(self.grid, cmap='hot', vmin=0, vmax=self.max_grains)
        ax.axis('off')

        puntos = self.centros
        punto_interseccion = self.punto_medio(self.marcar(puntos))

        # Calcular las coordenadas del cuadrado
        size_mg = 19 # Tamaño del cuadrado
        y_mg = int(punto_interseccion[0] - size_mg/ 2)
        x_mg = int(punto_interseccion[1] - size_mg / 2)

        size_mc = 9  # Tamaño del cuadrado
        y_mc = int(punto_interseccion[0] - size_mc / 2)
        x_mc = int(punto_interseccion[1] - size_mc / 2)

        # Dibujar el cuadrado
        rect1 = plt.Rectangle((x_mg, y_mg), size_mg, size_mg, linewidth=1.5, edgecolor='green', facecolor='none')
        rect2 = plt.Rectangle((x_mc, y_mc), size_mc, size_mc, linewidth=1.5, edgecolor='blue', facecolor='none')
        ax.add_patch(rect1)
        ax.add_patch(rect2)

        if save:
            plt.savefig(filename, bbox_inches='tight')
        plt.show()

    #Método para mostrar por pantalla la grila completa
    def show(self, save=False, filename="sandpile.png"):
        """
        Plot sandpile and/or save it to a file

        save - True to save the picture, False to only show it
        filename - name of the file where the sandpile picture will be saved
        """
        colors = [(0, 0, 0), (255, 0, 0), (255, 165, 0), (255, 255, 0)]  # Colors for heat scale

        plt.imshow(self.grid, cmap='hot', vmin=0, vmax=self.max_grains)
        plt.colorbar(ticks=range(self.max_grains))
        plt.axis('off')
        
        if save:
            plt.savefig(filename, bbox_inches='tight')
        plt.show()
    #Método que calcula la cantidad de arena que existe dentro de un cuadrante definido. tamaño_mg = tamaño matriz grande, tamaño_mc = tamaño matriz chica
    def sumar_matriz_cuadrado(self, tamano_mg = 19,tamano_mc = 9):

        centro = self.punto_medio(self.marcar(puntos)) #PROBLEMA: USA LOS PUNTOS NO DESPLAZADOS (SIN DELTA) -CORREGIDO-
        n = len(self.grid)
        m = len(self.grid[0])

        x1_mg = max(0, centro[1] - tamano_mg // 2) #(y1,x1) = (20,17)
        y1_mg = max(0, centro[0] - tamano_mg // 2)
        x2_mg = min(n - 1, centro[1] + tamano_mg // 2) #(y1,x1) = (17,23)
        y2_mg = min(m - 1, centro[0] + tamano_mg // 2)
        total_arena_mg = 0
        for i in range(y1_mg, y2_mg + 1):
            for j in range(x1_mg, x2_mg + 1):
                total_arena_mg += self.grid[i][j]

        x1_mc = max(0, centro[1] - tamano_mc // 2)
        y1_mc = max(0, centro[0] - tamano_mc // 2)
        x2_mc = min(n - 1, centro[1] + tamano_mc // 2)
        y2_mc = min(m - 1, centro[0] + tamano_mc // 2)
        total_arena_mc = 0
        for i in range(y1_mc, y2_mc + 1):
            for j in range(x1_mc, x2_mc + 1):
                total_arena_mc += self.grid[i][j]

        print("El valor de densidad de la celda promedio en el área circundante es:", round((total_arena_mg-total_arena_mc)/(tamano_mg**2-tamano_mc**2),2))
        print("El valor de densidad de la celda promedio del área de choque es:", round(total_arena_mc/tamano_mc**2,2))
        if (total_arena_mc/tamano_mc**2) < ((total_arena_mg-total_arena_mc)/(tamano_mg**2-tamano_mc**2)): 
            print("\nSI se cumple hipótesis (mayor densidad en área circundante que área de choque)")
        elif (total_arena_mc/tamano_mc**2) > ((total_arena_mg-total_arena_mc)/(tamano_mg**2-tamano_mc**2)):
            print("\nNO se cumple la hipótesis (menor densidad en área circundante que área de choque)")
    #Método que obtiene una lista de puntos y entrega los puntos adaptados a la nueva grilla.    
    def new_puntos(self, puntos_c):
        delta = int((self.cols - self.distancia) / 2) -1
        centros = np.copy(puntos_c)
        
        for i in centros:
            i[0] = i[0] + delta
            i[1] = i[1] + delta
        
        return centros
    #Método del nuevo tamaño de la grilla 
    def new_size(self):
        return self.cols
#Función que toma una grilla y una lista de colores y entrega una nueva grilla con los colores correspondientes.
def color_grid(grid, colors):
    new_grid = np.zeros((len(grid), len(grid[0]), 3), dtype=np.uint8)
    for i in range(len(new_grid)):
        for j in range(len(new_grid[0])):
            index = int(grid[i, j])  # Convertir el índice a entero
            new_grid[i, j] = colors[index]
    return new_grid

#Condicion para ejecutar la funcion principal
if __name__ == '__main__':
    pile = Sandpile(puntos_c = puntos)
    pile.set_sand(10000) # Cantidad de arena por punto
    pile.run()
    # pile.marcar(puntos_c = puntos) # Para ver donde están los centros
    pile.show(save = True, filename = "Sand_Test.png")
    pile.show_with_square()
    #pile.show(save = True, filename = "Sand_square.png")
    pile.sumar_matriz_cuadrado()
    puntos_animacion = pile.new_puntos(puntos)
    size_animacion = pile.new_size()

In [ ]:
#Clase usada principalmente para animar el proceso del modelo Sandpile
class Sandpile():
    def __init__(self, arr=None, puntos_c=None, size=10, max_sand=4):
        """
        arr - 2d array of values
        puntos_c - list of sandpile center points [Y_axis,X_axis]
        max_sand - max count of sandpile grains (must be div by 4)
        """
        #Umbral critico
        self.max_grains = max_sand

        #Variable donde guarda los puntos donde colocar la arena (lista)
        self.centros = puntos_c

        #Tamano de la grilla que se desea inciar
        self.size_grilla = size

        #Genera una grilla de un tamano deseado de puros 0 si es que no hay una existente
        if arr is None:
            self.rows = self.size_grilla
            self.cols = self.size_grilla
            self.grid = np.zeros((self.rows,self.cols), int)
            self.gridinicial = self.grid
        else:
            self.rows = len(arr)
            self.cols = len(arr[0])
            self.grid = np.array(arr)
            
    #Funcion que se encarga de repartir arenas a los vecinos dependiendo del valor critico
    def topple(self, elem_x, elem_y):
        #Analisa la casilla p
        p = self.grid[elem_x, elem_y]
        #variable de la cantidad de arena a repartir
        b = p // self.max_grains
        #Variable de la cantidad restante de arena que no puede repartir
        o = p % self.max_grains
        self.grid[elem_x, elem_y] = o

        # Reparte la arena a los vecinos1
        self.grid[elem_x-1, elem_y] += b
        self.grid[elem_x+1, elem_y] += b
        self.grid[elem_x, elem_y-1] += b
        self.grid[elem_x, elem_y+1] += b

        # border
        self.grid[0] = self.grid[-1] = 0
        self.grid[:, 0] = self.grid[:, -1] = 0
            
    #Funcion que coloca un numero de arena en un punto determinado
    def set_sand(self, number):
        #Lista de los puntos a colocar arena
        centros = self.centros
        
        #Coloca arena en los puntos
        for i in centros:
            self.grid[i[0]-1,i[1]-1] += number

    #Funcion que corre el modelo completo
    def run(self, number):
        from time import time
        
        #Creacion de la figura de animacion
        
        #Se coloca 3 como numero maximo de color mas alto
        self.grid[0,0] = 4
        fig = plt.figure(figsize=(5,5))
        grafico = plt.imshow(self.grid, cmap="hot")
        #Se elimina el previo 3 ya que no va con el modelo
        self.grid[0,0] = 0
        
        #Lista donde quedara guardado cada frame para la animacion
        lista = []
        
        #Variable para contar el tiempo que tarda el programa
        start_time = time()
        #Variable para contar las iteraciones o la cantidad de frame para usar
        iterations = 0
        #Llama a la funcion topple
        topple = self.topple
        where = np.where
        
        #Agregra a la lista el primer frame
        lista.append(np.copy(self.grid))
        #Variable de numero de granos totales por punto
        numero_granos = number
        
        #Cantidad de arena por poner en cada iteracion
        num_g = 8
        
        #Ciclo para colocar una cierta cantidad de arena respecto al total
        while numero_granos > 0:
            #Si la cantidad de arena restante es mayor, se coloca arena por iteracion
            if (numero_granos >= num_g):
                pile.set_sand(num_g)
                #Actualiza la arena restante
                numero_granos = numero_granos - num_g
            
            #Si la cantidad de arena restante es menor, se coloca lo sobrante
            if (numero_granos < num_g):
                pile.set_sand(numero_granos)
                #Actualiza la arena restante
                numero_granos = numero_granos - numero_granos
            
            #Ciclo que analisa el umbral critico y reparte arena    
            while np.max(self.grid) >= self.max_grains:
                elem_x, elem_y = where(self.grid >= self.max_grains)
                topple(elem_x, elem_y)
            
            #Guarda cada final de analisis como frame
            lista.append(np.copy(self.grid))
            #Aumenta una iteracion luego de terminado el proceso
            iterations += 1
            
        #Imprime en pantalla la cantidad de iteraciones y cuanto duro el programa
        print("--- %d iterations %s seconds ---" % (iterations, time() - start_time))
        
        #Genera un formato grafico para crear la animacion
        def update(frame):
            grafico.set_data(lista[frame])
            return grafico
        
        #Variable de animacion
        ani = animation.FuncAnimation(fig, 
                                    update, 
                                    frames   = iterations + 1,
                                    interval = 10
                                    );
        
        #Guarda la animacion en mp4, requerido instalar ffmpeg - EN CASO DE ERROR, INVALIDAR/ELIMINAR LINEA SIGUIENTE
        ani.save('animacion.mp4', dpi=60, writter="ffmpeg")
        
        #Despliega en pantalla la animacion del modelo
        display(HTML(ani.to_jshtml()))

    def __add__(self, other):
        result = Sandpile(rows = self.rows, cols = self.cols)
        try:
            result.grid = self.grid + other.grid
            return result.run()
        except ValueError:
            print("ValueError: sandpile grid sizes must match")

if __name__ == '__main__':
    #Genera el modelo en base a los parametro del codigo anterior, tamaño grilla y puntos de arena desplazados
    pile = Sandpile(puntos_c = puntos_animacion, size = size_animacion)
    #Corre el programa y como parametro es la cantidad de arena por punto
    pile.run(10000)